In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters= 10000
learning_rate = 3e-4


cuda


In [3]:
with open("Wizard_of_oz.txt",'r',encoding='utf-8') as file:
    text = file.read()
chars = sorted(set(text))
print(chars)
vocabulary_len = len(chars)


['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [4]:
string_ito_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

encode = lambda s:[string_ito_int[ch] for ch in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [5]:
encoded_text= encode("hello")
decode_text = decode(encoded_text)

print("This is encoded:", encoded_text) ## testing
print("This is decoded:", decode_text) ## testing

This is encoded: [63, 60, 67, 67, 70]
This is decoded: hello


In [6]:

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([82, 46, 63, 60,  1, 42, 73, 70, 65, 60, 58, 75,  1, 33, 76, 75, 60, 69,
        57, 60, 73, 62,  1, 60, 28, 70, 70, 66,  1, 70, 61,  1, 30, 70, 73, 70,
        75, 63, 80,  1, 56, 69, 59,  1, 75, 63, 60,  1, 49, 64, 81, 56, 73, 59,
         1, 64, 69,  1, 41, 81,  0,  1,  1,  1,  1,  0, 46, 63, 64, 74,  1, 60,
        57, 70, 70, 66,  1, 64, 74,  1, 61, 70, 73,  1, 75, 63, 60,  1, 76, 74,
        60,  1, 70, 61,  1, 56, 69, 80, 70, 69])


In [7]:
new_n = int(0.8*len(data))
train_split = data[:new_n]
val_split = data[new_n:]

In [8]:
def get_batch(split):
    data = train_split if split == 'train' else val_split
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y 


x, y = get_batch('train')
print("inputs")
print(x)
print("targets")
print(y)

tensor([121713,  74430, 178831, 172090])


inputs
tensor([[76, 73,  1, 75, 73, 56, 77, 60],
        [60, 80,  1, 58, 73, 76, 74, 63],
        [56, 71, 60, 59,  1, 75, 70, 70],
        [ 1, 78, 70, 69, 59, 60, 73, 64]], device='cuda:0')
targets
tensor([[73,  1, 75, 73, 56, 77, 60, 67],
        [80,  1, 58, 73, 76, 74, 63, 60],
        [71, 60, 59,  1, 75, 70, 70,  1],
        [78, 70, 69, 59, 60, 73, 64, 69]], device='cuda:0')


In [16]:

# Bigram Language Model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Each token directly maps to a vector of logits for the next token
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        # index and targets are both (B, T) tensors
        logits = self.token_embedding_table(index)  # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)          # flatten to (B*T, C)
            targets = targets.view(B * T)          # flatten to (B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index: (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)       # (B, T, C)
            
            # focus only on the last time step
            logits = logits[:, -1, :]                # (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)        # (B, C)
            
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1)         # (B, T+1)
        return index


model = BigramLanguageModel(vocabulary_len)
m = model.to(device)

# start with a single token
context = torch.zeros((1, 1), dtype=torch.long, device=device)

# generate 100 new tokens
generated_tokens = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_tokens)


RDtbQ&*Bt_? g5NLY:KFp-5P8z-E4zEb_ax[6zoO[6UZ#&lH3-Q89aY6";*Aj*VbOd]DBseW(
THEQJXPI _1j#duKXQ&*DtM﻿u﻿!vI-w6pZ,&p-EL3J _K(vB﻿KbQ;O﻿.&X!)i9Z.sIT0vs[d]qawAOb﻿p86v GD.YQ;EIvs2WeU#jVWbhZDR&Jort]:Qt]kB
8qSTTlfOZ_P8G77N7(BYK1O-4?C_GS]do;RQm﻿.GrFitCHHx(J0QF
68E]qCmm&!jkpF7L3-Z3"bS&g5/sU'XhQJGDggmGp)9aC;﻿u3;GDY ua13Wzo6gTQ8C﻿J"lfbOjVmZ.R_3"ks6pQ&ekAa7_o"6rxC&Wd092RuQb12WN3-H3D.Uhk-ETBiZ_0kW(Bh-x!1wiY(_Kj-2sD(YxrdFm92fja.!1k)aDLp"R#B;F!3D3Ki_?n_jYoXhA]csun!01OXXskD:xA2OIA4!Npqn
tCA*G"c:O4mNW(HGhxSa.b)k)q,h


In [9]:
## input and output mapping


x= train_split[:block_size]
y= train_split[1:block_size+1]

for i in range(block_size):
    content = x[:i+1]
    target = y[i]
    
    print(f"When th input is{content} the target is {target}")

When th input istensor([82]) the target is 46
When th input istensor([82, 46]) the target is 63
When th input istensor([82, 46, 63]) the target is 60
When th input istensor([82, 46, 63, 60]) the target is 1
When th input istensor([82, 46, 63, 60,  1]) the target is 42
When th input istensor([82, 46, 63, 60,  1, 42]) the target is 73
When th input istensor([82, 46, 63, 60,  1, 42, 73]) the target is 70
When th input istensor([82, 46, 63, 60,  1, 42, 73, 70]) the target is 65
